# Using statistic regression methods for analysing different features' affect on amount of children

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as ss

In [3]:
df = pd.read_csv('botswana.tsv', sep='\t', header=0)

In [4]:
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


columnt info:

ceb = amount of children

age = age of woman when she had her first children

educ = years of getting an education

religion = religion

idlnchld = ideal amount of children (in her opinion)

knowmeth = if she knows about contraception methods

usemeth = if she uses contraception methods

evermarr = was she ever married

agefm = how many years she had been married

heduc = how many years her husband had in getting an education

urban = if she lives in a city

electric = if she uses electric devices often

radio = if she listens to radio

tv = if she watches tv

bycicle = if she rides bicycle

In [5]:
df.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

In [6]:
print(df.shape)
print((df.dropna()).shape)

(4361, 15)
(1834, 15)


Let's get a rid of "NaN" fields by adding special noan columns to save integrity of our data

In [7]:
df['nevermarr'] = df['agefm'].apply(lambda x: 1 if pd.isna(x) else 0)

In [8]:
df.drop(['evermarr'], axis=1, inplace=True)

In [9]:
df['agefm'].fillna(0, inplace=True)

In [10]:
for i in range(len(df['heduc'])):
    if df['nevermarr'][i]==1 and pd.isnull(df['heduc'][i]):
        df['heduc'][i] = -1

C:\Users\Aqice\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
df['idlnchld_noans'] = df['idlnchld'].apply(lambda x: 1 if pd.isna(x) else 0)
df['idlnchld'].fillna(-1, inplace=True)
df['heduc_noans'] = df['heduc'].apply(lambda x: 1 if pd.isna(x) else 0)
df['heduc'].fillna(-2, inplace=True)
df['usemeth_noans'] = df['usemeth'].apply(lambda x: 1 if pd.isna(x) else 0)
df['usemeth'].fillna(-1, inplace=True)

In [15]:
df.dropna(inplace=True)

Now we can use statistic linear regression to show how separate feature affects on final "ceb" column

In [17]:
import statsmodels.formula.api as smf
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=df)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 30 Aug 2018   Prob (F-statistic):               0.00
Time:                        23:14:55   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [18]:
import statsmodels.stats.api as sms
print('p = {}'.format(sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]))

p = 1.1452927633439797e-225


C:\Users\Aqice\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  


Some features are odd and Breusch–Pagan test shows a very small p-value

We should drop some features and use "HC1" fit parameter to normalize coef values

In [19]:
df_new = df.drop(['religion', 'radio', 'tv'], axis=1)

In [20]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=df_new)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())
print('p = {}'.format(sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]))

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 30 Aug 2018   Prob (F-statistic):               0.00
Time:                        23:15:42   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

C:\Users\Aqice\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  after removing the cwd from sys.path.


In [21]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


This model seems to be better. Let's get a rid of "usemeth" columns also

We need to delete "usemeth_noans" columns also because we added some special values (-1) which depend on parents' column

In [22]:
df_new2 = df_new.drop(['usemeth', 'usemeth_noans'], axis=1)
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans', data=df_new2)
fitted = m2.fit(cov_type='HC1')
print(fitted.summary())
f, p, k1 = m2.fit().compare_f_test(m3.fit())
print('p = {}'.format(p))

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 30 Aug 2018   Prob (F-statistic):               0.00
Time:                        23:16:06   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

Coef columns shows what features and how affect on amount of children